## Segmenting and Clustering Neighborhoods in Toronto

### Question 1

In [29]:
!pip install folium
!pip install geocoder

import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


import geocoder
import requests
import json
import folium

from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
print('Ready to go!')

Ready to go!


In [30]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"}
r = requests.get(url, headers = header)
table = pd.read_html(r.text)
toronto_df = pd.DataFrame(table[0])
toronto_df.columns=['PostalCode','Borough','Neighborhood']

In [31]:
toronto_df_dropna = toronto_df[toronto_df.Borough!='Not assigned'].reset_index(drop= True)
toronto_df_cleaned = toronto_df_dropna.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
na_neigh_rows = toronto_df_cleaned.Neighborhood == 'Not assigned'
toronto_df_cleaned.loc[na_neigh_rows, 'Neighborhood'] = toronto_df_cleaned.loc[na_neigh_rows, 'Borough']
toronto_df_cleaned.shape

(103, 3)

### Question 2

In [34]:
!wget -q -O "toronto_coordinates.csv" https://cocl.us/Geospatial_data
coors = pd.read_csv('toronto_coordinates.csv')
print('Coordinates dataframe created!')

Coordinates dataframe created!


In [40]:
toronto_df_temp = toronto_df_cleaned.set_index('PostalCode')
coors_temp = coors.set_index('Postal Code')
toronto_df_coors = pd.concat([toronto_df_temp, coors_temp], axis=1,join='inner')

toronto_df_coors.index.name = 'PostalCode'
toronto_df_coors.reset_index(inplace=True)

print(toronto_df_coors.shape)
toronto_df_coors.head(12)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Question 3

In [69]:
location = geolocator.geocode('New York, New York')
ny_latitude = location.latitude
ny_longitude = location.longitude
print('New York coordinate: {},{}'.format(ny_latitude, ny_longitude))
geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode('Toronto, Ontario')
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('Toronto coordinate: {},{}'.format(latitude, longitude))

New York coordinate: 40.7127281,-74.0060152
Toronto coordinate: 43.6534817,-79.3839347


## Map of Toronto with Boroughs markers on top

In [70]:
toronto_map = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(toronto_df_coors['Latitude'], toronto_df_coors['Longitude'], toronto_df_coors['PostalCode'], toronto_df_coors['Borough'], toronto_df_coors['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
toronto_map